In [35]:
%matplotlib inline

import os
import sys
import json

import numpy as np
import pandas as pd
from pandas.io.stata import StataReader

from matplotlib import pyplot as plt
from IPython.display import display
import seaborn as sns
sns.set()

from sklearn.model_selection import train_test_split

# Add our local functions to the path
sys.path.append(os.path.join(os.pardir, 'src'))
from data import load_data
from features import process_features

ModuleNotFoundError: No module named 'data'

In [31]:
print(os.pardir)

..


In [5]:
# Read stata data into dataframe:
survey = pd.read_stata(r"C:\Users\micha\Documents\GitHub\mkp_code\Institute of Data Course\Project 3\data\MWI_2010_IHS-III_v01_M_v01_A_ML_Stata\MWI_2010_community.dta")

In [7]:
survey.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 768 entries, 0 to 767
Data columns (total 16 columns):
 #   Column           Non-Null Count  Dtype   
---  ------           --------------  -----   
 0   ea_id            768 non-null    object  
 1   com_roadtype     768 non-null    category
 2   com_vehicles     768 non-null    category
 3   com_bus          768 non-null    category
 4   com_urbancenter  768 non-null    category
 5   com_dailymrkt    768 non-null    category
 6   com_weeklymrkt   768 non-null    category
 7   com_postoffice   768 non-null    category
 8   com_publicphone  768 non-null    category
 9   com_distprimary  679 non-null    category
 10  com_classrooms   767 non-null    category
 11  com_schoolelec   768 non-null    category
 12  com_medicines    768 non-null    category
 13  com_clinic       768 non-null    category
 14  com_distclinic   566 non-null    category
 15  com_bank         768 non-null    category
dtypes: category(15), object(1)
memory usage: 45.

### Source: <br> The World Bank data preparation notebook available @ GitHub: <br> https://github.com/worldbank/ML-classification-algorithms-poverty/tree/master/notebooks


In [12]:
# This function loads the sample stata file:
def load_stata_file(filepath, 
                    index_cols, 
                    update_education=False, 
                    drop_minornans=False, 
                    drop_unlabeled=False):
    """ Load data and metadata from Stata file"""
    data = pd.read_stata(filepath, convert_categoricals=False).set_index(index_cols)

    with StataReader(filepath) as reader:
        reader.value_labels()
            
        mapping = {col: reader.value_label_dict[t] for col, t in 
                   zip(reader.varlist, reader.lbllist)
                   if t in reader.value_label_dict}
        
        # manually update some specific columns
        # in the Malawi dataset
        if update_education and 'ind_educ09' in mapping:
            mapping['ind_educ09'][13] = "Primary - " + mapping['ind_educ09'][13]
            mapping['ind_educ09'][14] = "Primary - " + mapping['ind_educ09'][14]
            
            mapping['ind_educ09'][23] = "Secondary - " + mapping['ind_educ09'][23]
            mapping['ind_educ09'][24] = "Secondary - " + mapping['ind_educ09'][24]
        
        data.replace(mapping, inplace=True)
        
        # convert the categorical variables into
        # the category type
        for c in data.columns:
            if c in mapping:
                data[c] = data[c].astype('category')
                        
        # drop records with only a few nans
        if drop_minornans: 
            nan_counts = (data.applymap(pd.isnull)
                          .sum(axis=0)
                          .sort_values(ascending=False))
            nan_cols = nan_counts[(nan_counts > 0) & (nan_counts < 10)].index.values
            data = data.dropna(subset=nan_cols)
        # drop unlabeled categorical values
        def find_unlabeled(x):
            if x.name in mapping.keys():
                return [val if (val in mapping[x.name].values() or pd.isnull(val)) 
                        else 'UNLABELED' for val in x]
            else:
                return x
            
        data = data.apply(find_unlabeled)
        data = data[~data.applymap(lambda x: x == "UNLABELED").any(axis=1)]
        
        # read the actual questions that were asked for reference
        questions = reader.variable_labels()
        
    return data, questions

In [11]:
# Load Malawi household data
filepath = load_data.MWI_HOUSEHOLD
mwi_hhold, mwi_hhold_questions = load_stata_file(filepath, 
                                                 ['hid'], 
                                                 drop_minornans=True, 
                                                 drop_unlabeled=True)

s = 'Malawi household data has {:,} rows and {:,} columns'
print(s.format(mwi_hhold.shape[0], mwi_hhold.shape[1]))
s = 'Percent poor: {:0.1%} \tPercent non-poor: {:0.1%}'
print(s.format(mwi_hhold.poor.value_counts(normalize=True).loc['Poor'], 
               mwi_hhold.poor.value_counts(normalize=True).loc['Non-poor']))
mwi_hhold.head()

NameError: name 'load_data' is not defined